# Experiments

## Colab setup

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip install ogb
!pip install grandiso

Loading drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Mount the current directory
%cd /content/drive/My\ Drive//CS159_project/Graph_homomorphism/graph_homomorphism

### imports

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.transforms import BaseTransform
from torch_geometric.datasets import TUDataset, ZINC
#import torch_geometric.utils as uts
#from torch_geometric.utils import remove_self_loops, to_undirected

import numpy as np
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm

In [14]:
# Graph utilities
from graph_encoding.graphencoding import grandEmbedding 


In [15]:
G = grandEmbedding(None)

Datasets:

In [5]:
from ogb.graphproppred import PygGraphPropPredDataset